<h1>네이버 부동산 지도 크롤링

### 1. 용원동 크롤링 및 데이터 정제

<h4>1) 라이브러리 호출

In [1]:
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용

<h4>2) 셀레니움 옵션지정

In [2]:
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지

<h4>3) 셀레니움 드라이버 실행 세팅

In [66]:
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

#부산신항 주변 → 경상남도 > 창원시 진해구 > 용원동
targetUrl = "https://new.land.naver.com/offices?ms=35.0920766,128.8153858,15&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

number_idx = 1
#좌측바 메뉴 개수를 세어보니 20개임. 따라서 반복문 지정을 20까지로 설정
for i in range(2,21):
    number_idx +=1  #순번
    idx_list.append(number_idx)
    leftMenu = myDriver.find_element(By.XPATH, value ='''//*[@id="listContents1"]/div/div/div[1]/div['''+ str(i) +''']''').click() #좌측바 메뉴별 XPATH
    time.sleep(4)
    #BeautifulSoup4 
    webCurrentStatus = myDriver.page_source   #좌측바 메뉴 클릭후 생성된 사이드 바가 표시된 상태 자체를 변수로 지정
    myParser = BeautifulSoup(webCurrentStatus, "html.parser") #위 코드상태를 html로 파싱
    side_Menu = myParser.select("div.detail_contents.is-article") #크롤링 영역 지정 
    for i in side_Menu: #위 코드에서 Select을 사용하였기에 출력이 리스트 타입이므로 반복문 진행
        #↓추출하고자하는 데이터별 태그+클래스네임을 활용하여 데이터 추출
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc= i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle #주소 + 매물명 합침
        #↓위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

https://new.land.naver.com/offices?ms=35.0920766,128.8153858,15&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165 접속완료


<h4> 4) 데이터프레임 전처리(1)

In [65]:
#데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

storagePlace

for i in range(0,19):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    
    if storagePlace.loc[i,"면적(㎡)"].split(":"):
       storagePlace.loc[i,"면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split(":")[2]
    
    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")


storagePlace['보증금'] = storagePlace['보증금'].astype(int)
storagePlace['월세'] = storagePlace['월세'].astype(int)
storagePlace['월세'] = storagePlace['월세'] * 10000

storagePlace.head(2)       

ValueError: All arrays must be of the same length

In [10]:
storagePlace.to_csv('../data/storagePlaceYong0.1.csv')

<h4> 5) 용원동데이터프레임 전처리(2) - 구분 추가, 값 이상한 행 삭제 등 

In [38]:
# 용원동 크롤링 저장한 파일 불러오기
storagePlaceYong= pd.read_csv('../data/storagePlaceYong0.1.csv')
storagePlaceYong.head(2)

,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소
0,0,2,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가
1,1,3,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가


In [39]:
# 매매 타입 유형의 값을 매매가라는 새로운 칼럼에 기입 및 값 변경
sell = storagePlaceYong[storagePlaceYong['타입'] == '매매']
storagePlaceYong.loc[sell.index, '매매가'] = sell['가격']
storagePlaceYong['매매가'].fillna('-', inplace=True)
storagePlaceYong.head(2)

,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소,매매가
0,0,2,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-
1,1,3,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-


In [40]:
# 보증금, 월세 값 0 대신 -으로 변경
storagePlaceYong['보증금'] = storagePlaceYong['보증금'].replace(0, '-')
storagePlaceYong['월세'] = storagePlaceYong['월세'].replace(0, '-')

In [41]:
storagePlaceYong.head(2)

,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소,매매가
0,0,2,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-
1,1,3,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-


In [42]:
# 구분 컬럼 추가
storagePlaceYong['구분']='용원동'
storagePlaceYong.head(2)

,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소,매매가,구분
0,0,2,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-,용원동
1,1,3,단지내상가 신한주상가4층,월세,256.68㎡/153.05㎡,256.68,153.05,-,"1,000/160",1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,-,용원동


In [43]:
# 필요 칼럼만 불러오기
storagePlaceYong = storagePlaceYong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
storagePlaceYong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금,월세,주소,명칭
0,2,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층
1,3,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층


In [44]:
# 칼럼 이름 변경
storagePlaceYong = storagePlaceYong.rename(columns={'명칭': '기타(세부주소)'})
storagePlaceYong = storagePlaceYong.rename(columns={'보증금': '보증금 (단위: 만원)'})
storagePlaceYong = storagePlaceYong.rename(columns={'월세': '월세(단위:만원)'})
storagePlaceYong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금 (단위: 만원),월세(단위:만원),주소,기타(세부주소)
0,2,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층
1,3,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층


In [45]:
# 타입별로 분류하기
storagePlaceYong = storagePlaceYong.sort_values(by='타입', ascending=False)
storagePlaceYong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금 (단위: 만원),월세(단위:만원),주소,기타(세부주소)
0,2,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층
2,4,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층


In [46]:
# 순번 열 자동화
storagePlaceYong.reset_index(drop=True, inplace=True)
storagePlaceYong['순번'] = storagePlaceYong.index + 1
storagePlaceYong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금 (단위: 만원),월세(단위:만원),주소,기타(세부주소)
0,1,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층
1,2,용원동,월세,256.68,153.05,-,-,1000,1600000,경상남도 창원시 진해구 용원동 단지내상가 신한주상가,단지내상가 신한주상가4층


In [47]:
# 전처리 끝난 용원동 파일 저장하기
storagePlaceYong.to_csv('../data/yongwondong0.1')

### 2. 천성동 웹크롤링 및 데이터 정제

In [48]:
# 라이브러리 로드
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
# 셀레니움 옵션 지정
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

In [49]:
#천성동
targetUrl = "https://new.land.naver.com/offices?ms=35.0361166,128.8267131,15&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

https://new.land.naver.com/offices?ms=35.0361166,128.8267131,15&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165 접속완료


In [50]:
# 천성동 리스트 길이 확인
total_items = len(myDriver.find_elements(By.XPATH, '//*[@id="listContents1"]/div/div/div[1]/div'))
total_items

12

In [51]:
#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

In [55]:
number_idx = 0

# Iterate through all items
for i in range(1, total_items + 1):
    # 순번
    number_idx += 1  
    idx_list.append(number_idx)

    # 동적으로 XPath를 생성
    element_xpath = f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]'
    
    # 좌측바 메뉴 클릭
    leftMenu = myDriver.find_element(By.XPATH, value=element_xpath).click()
    time.sleep(4)
    
    # 나머지 코드
    webCurrentStatus = myDriver.page_source
    myParser = BeautifulSoup(webCurrentStatus, "html.parser")
    side_Menu = myParser.select("div.detail_contents.is-article")
    
    for i in side_Menu:
        # ↓ try 블록 내에서 데이터를 추출하는 코드
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle  # 주소 + 매물명 합침
        
        # ↓ 위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00528C93+51395]
	(No symbol) [0x00495EF1]
	(No symbol) [0x0034E13A]
	(No symbol) [0x003325F2]
	(No symbol) [0x003A9B2B]
	(No symbol) [0x003B9E66]
	(No symbol) [0x003A3C26]
	(No symbol) [0x0037C629]
	(No symbol) [0x0037D40D]
	GetHandleVerifier [0x008A6453+3711107]
	GetHandleVerifier [0x008E583A+3970154]
	GetHandleVerifier [0x008E0B28+3950424]
	GetHandleVerifier [0x005D9C99+776393]
	(No symbol) [0x004A16C4]
	(No symbol) [0x0049C5A8]
	(No symbol) [0x0049C759]
	(No symbol) [0x0048DD80]
	BaseThreadInitThunk [0x757800C9+25]
	RtlGetAppContainerNamedObjectPath [0x77407B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77407AEE+238]


In [54]:
# 데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

for i in range(len(storagePlace)):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
        
    if ":" in storagePlace.loc[i, "면적(㎡)"]:
        storagePlace.loc[i, "면적(㎡)"] = storagePlace.loc[i, "면적(㎡)"].split(":")[2]
    else:
        storagePlace.loc[i, "면적(㎡)"] = ""  # Handle the case where ":" is not present

    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")

    storagePlace.loc[i, "보증금"] = pd.to_numeric(storagePlace.loc[i, "보증금"], errors='coerce')
    storagePlace.loc[i, "월세"] = pd.to_numeric(storagePlace.loc[i, "월세"], errors='coerce')

storagePlace.head(2)

ValueError: All arrays must be of the same length

In [ ]:
# 천성동 웹크롤링 파일 저장
storagePlace.to_csv('../data/Chunsundong.csv')

In [57]:
# 천성동 저장한 파일 불러오기
Chunsundong = pd.read_csv('../data/Chunsundong.csv')
Chunsundong.head(2)

,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소
0,0,1,전,매매,721㎡,-,-,721,10억,0,0,부산시 강서구 천성동 전
1,1,2,전,매매,721㎡,-,-,721,10억,0,0,부산시 강서구 천성동 전


In [58]:
# 매매 타입 유형은 가격값을 매매가라는 새로운 칼럼에 기입 및 값 변경
sell = Chunsundong[Chunsundong['타입'] == '매매']
Chunsundong.loc[sell.index, '매매가'] = sell['가격']
Chunsundong['매매가'].fillna('-', inplace=True)

In [59]:
# 구분에 천성동 값 기입
Chunsundong['구분']='천성동'

In [60]:
# 필요 컬럼만 불러오기
Chunsundong = Chunsundong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
Chunsundong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금,월세,주소,명칭
0,1,천성동,매매,-,-,721,10억,0,0,부산시 강서구 천성동 전,전
1,2,천성동,매매,-,-,721,10억,0,0,부산시 강서구 천성동 전,전


In [61]:
# 칼럼값 명칭 변경하기
Chunsundong = Chunsundong.rename(columns={'명칭': '기타(세부주소)'})
Chunsundong = Chunsundong.rename(columns={'보증금': '보증금 (단위: 만원)'})
Chunsundong = Chunsundong.rename(columns={'월세': '월세(단위:만원)'})
Chunsundong.head(2)

,순번,구분,타입,계약면적(㎡),전용면적(㎡),대지면적(㎡),매매가,보증금 (단위: 만원),월세(단위:만원),주소,기타(세부주소)
0,1,천성동,매매,-,-,721,10억,0,0,부산시 강서구 천성동 전,전
1,2,천성동,매매,-,-,721,10억,0,0,부산시 강서구 천성동 전,전


In [62]:
# 정제 파일 저장
Chunsundong.to_csv('../data/chunsundong0.1.csv')

### 3. 명지동 웹크롤링 및 데이터 정제

* 웹크롤링 부분 위와 반복

In [64]:
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

In [ ]:
#명지동
targetUrl = "https://new.land.naver.com/offices?ms=35.108082,128.92635,16&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

In [ ]:
total_items = len(myDriver.find_elements(By.XPATH, '//*[@id="listContents1"]/div/div/div[1]/div'))
total_items

In [ ]:
#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

In [ ]:
number_idx = 0

# Iterate through all items
for i in range(1, total_items + 1):
    # 순번
    number_idx += 1  
    idx_list.append(number_idx)

    # 동적으로 XPath를 생성
    element_xpath = f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]'
    
    # 좌측바 메뉴 클릭
    leftMenu = myDriver.find_element(By.XPATH, value=element_xpath).click()
    time.sleep(4)
    
    # 나머지 코드
    webCurrentStatus = myDriver.page_source
    myParser = BeautifulSoup(webCurrentStatus, "html.parser")
    side_Menu = myParser.select("div.detail_contents.is-article")
    
    for i in side_Menu:
        # ↓ try 블록 내에서 데이터를 추출하는 코드
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle  # 주소 + 매물명 합침
        
        # ↓ 위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

In [ ]:
# 데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

In [ ]:
#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

storagePlace

In [ ]:
for i in range(len(storagePlace)):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
        
    if ":" in storagePlace.loc[i, "면적(㎡)"]:
        storagePlace.loc[i, "면적(㎡)"] = storagePlace.loc[i, "면적(㎡)"].split(":")[2]
    else:
        storagePlace.loc[i, "면적(㎡)"] = ""  # Handle the case where ":" is not present

    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")

    storagePlace.loc[i, "보증금"] = pd.to_numeric(storagePlace.loc[i, "보증금"], errors='coerce')
    storagePlace.loc[i, "월세"] = pd.to_numeric(storagePlace.loc[i, "월세"], errors='coerce')

storagePlace['보증금'].fillna(0, inplace=True)

# Convert columns to integers
storagePlace['보증금'] = storagePlace['보증금'].astype(int)
storagePlace['월세'] = storagePlace['월세'].astype(int)
#storagePlace['월세'] = storagePlace['월세'] * 10000

In [ ]:
# 크롤링 파일 저장
storagePlace
storagePlace.to_csv('../jupyter/명지동0.3.csv')

In [67]:
# 저장한 크롤링 파일 불러오기
myungjidong = pd.read_csv('../data/명지동0.3.csv')
myungjidong.head(2)

,Unnamed: 0.1,Unnamed: 0,순번,명칭,타입,면적(㎡),계약면적(㎡),전용면적(㎡),대지면적(㎡),가격,보증금,월세,주소
0,0,0,1,일반상가 에코팰리스8층,월세,NaN,-,-,NaN,"2,000/80",2000,80,부산시 강서구 명지동 일반상가 에코팰리스
1,1,1,2,일반상가2층,월세,224.7㎡/139.8㎡,224.7,139.8,-,"3,000/180",3000,180,부산시 강서구 명지동 일반상가


In [ ]:
# 컬럼 데이터 오류 값 삭제
for i in range(len(myungjidong)):
    if pd.isna(myungjidong.loc[i, '면적(㎡)']):
        myungjidong = myungjidong.drop(i)
    elif pd.isna(myungjidong.loc[i, '대지면적(㎡)']):
        myungjidong = myungjidong.drop(i)
    elif myungjidong.loc[i, '전용면적(㎡)'] == '연 면적':
        myungjidong = myungjidong.drop(i)
    elif myungjidong.loc[i, '가격'] == '-':
        myungjidong = myungjidong.drop(i)
       
myungjidong = myungjidong.reset_index(drop=True)
myungjidong.head(2)

In [ ]:
# 매매가 칼럼 추가하고 매매 타입 유형에 대해 가격 칼럼 값을 매매가에 기입
sell = myungjidong[myungjidong['타입'] == '매매']

myungjidong.loc[sell.index, '매매가'] = sell['가격']
myungjidong['매매가'].fillna('-', inplace=True)
myungjidong.head(2)

In [ ]:
# 보증금, 월세 값 0 대신 -으로 변경
myungjidong['보증금'] = myungjidong['보증금'].replace(0, '-')
myungjidong['월세'] = myungjidong['월세'].replace(0, '-')
myungjidong.head(2)

In [ ]:
for i in range(1, 11):
    myungjidong['보증금'] = myungjidong['보증금'].replace(i, f'{i}0000')
    
myungjidong.head(2)

In [ ]:
# 필요한 칼럼만 가져오기
myungjidong = myungjidong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
myungjidong.head()

In [ ]:
# 명칭 칼럼 이름 변경
myungjidong = myungjidong.rename(columns={'명칭': '기타(세부주소)'})
myungjidong = myungjidong.rename(columns={'보증금': '보증금 (단위: 만원)'})
myungjidong = myungjidong.rename(columns={'월세': '월세(단위:만원)'})
myungjidong.head()

In [ ]:
# 타입별 정렬하기
myungjidong = myungjidong.sort_values(by='타입', ascending=False)
myungjidong.head()

In [ ]:
# 순번 자동화
myungjidong.reset_index(drop=True, inplace=True)
myungjidong['순번'] = myungjidong.index + 1
myungjidong.head()

In [ ]:
# 파일 저장
myungjidong.to_csv('../data/myungjidong0.1')

### 4. 신호동 크롤링 및 데이터 정제

In [ ]:
# 필요한 라이브러리 로드 및 옵션 지정
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지

In [ ]:
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

In [ ]:
#신호동
targetUrl = "https://new.land.naver.com/offices?ms=35.0674,128.8751,16&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

In [ ]:
#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

In [ ]:
total_items = len(myDriver.find_elements(By.XPATH, '//*[@id="listContents1"]/div/div/div[1]/div'))
total_items

In [ ]:
number_idx = 0

# Iterate through all items
for i in range(1, total_items + 1):
    # 순번
    number_idx += 1  
    idx_list.append(number_idx)

    # 동적으로 XPath를 생성
    element_xpath = f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]'
    
    # 좌측바 메뉴 클릭
    leftMenu = myDriver.find_element(By.XPATH, value=element_xpath).click()
    time.sleep(4)
    
    # 나머지 코드
    webCurrentStatus = myDriver.page_source
    myParser = BeautifulSoup(webCurrentStatus, "html.parser")
    side_Menu = myParser.select("div.detail_contents.is-article")
    
    for i in side_Menu:
        # ↓ try 블록 내에서 데이터를 추출하는 코드
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle  # 주소 + 매물명 합침
        
        # ↓ 위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

In [ ]:
# 데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

In [ ]:
#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

storagePlace

In [ ]:
for i in range(len(storagePlace)):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
        
    if ":" in storagePlace.loc[i, "면적(㎡)"]:
        storagePlace.loc[i, "면적(㎡)"] = storagePlace.loc[i, "면적(㎡)"].split(":")[2]
    else:
        storagePlace.loc[i, "면적(㎡)"] = ""  # Handle the case where ":" is not present

    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")

    storagePlace.loc[i, "보증금"] = pd.to_numeric(storagePlace.loc[i, "보증금"], errors='coerce')
    storagePlace.loc[i, "월세"] = pd.to_numeric(storagePlace.loc[i, "월세"], errors='coerce')

In [ ]:
storagePlace.head(2)

In [ ]:
storagePlace['보증금'].fillna(0, inplace=True)

# Convert columns to integers
storagePlace['보증금'] = storagePlace['보증금'].astype(int)
storagePlace['월세'] = storagePlace['월세'].astype(int)
storagePlace['월세'] = storagePlace['월세'] * 10000

# Loop to handle the specific case where '보증금' is 4
for i in range(len(storagePlace)):
    if storagePlace.loc[i, "보증금"] == 4:
        storagePlace.loc[i, "보증금"] = storagePlace.loc[i, "보증금"] * 100000000

storagePlace.head(2)

In [ ]:
# 크롤링 파일 저장
shinhodong.to_csv('../data/shinhodong0.1')

In [ ]:
# 크롤링 파일 불러오기
shinhodong = pd.read_csv('../data/신호동0.1.csv')

In [ ]:
# 면적 컬럼에 값 상이한 것 삭제
for i in range(len(shinhodong)):
    if shinhodong['전용면적(㎡)'][i] == '연 면적':
        shinhodong = shinhodong.drop(i)
       
shinhodong = shinhodong.reset_index(drop=True)
shinhodong.head(2)

In [ ]:
# 매매가 칼럼 추가하고 매매 타입 유형에 대해 가격 칼럼 값을 매매가에 기입
sell = shinhodong[shinhodong['타입'] == '매매']

shinhodong.loc[sell.index, '매매가'] = sell['가격']
shinhodong['매매가'].fillna('-', inplace=True)
shinhodong.head(2)

In [ ]:
# 보증금, 월세 값 0 대신 -으로 변경
shinhodong['보증금'] = shinhodong['보증금'].replace(0, '-')
shinhodong['월세'] = shinhodong['월세'].replace(0, '-')
shinhodong.head(2)

In [ ]:
# 구분 컬럼 추가
shinhodong['구분']='신호동'
shinhodong.head(2)

In [ ]:
# 필요한 칼럼만 가져오기
shinhodong = shinhodong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
shinhodong.head()

In [ ]:
# 보증금 값 변경 : 1억을 만원 단위로 표시
shinhodong['보증금'] = shinhodong['보증금'].replace(1, '10000')
shinhodong.head(2)

In [ ]:
# 명칭 칼럼 이름 변경
shinhodong = shinhodong.rename(columns={'명칭': '기타(세부주소)'})
shinhodong = shinhodong.rename(columns={'보증금': '보증금 (단위: 만원)'})
shinhodong = shinhodong.rename(columns={'월세': '월세(단위:만원)'})
shinhodong.head(2)

In [ ]:
# 타입별 분류
shinhodong_sorted = shinhodong.sort_values(by='타입', ascending=False)
shinhodong_sorted.head()

In [ ]:
# 순번 열 자동화
shinhodong_sorted.reset_index(drop=True, inplace=True)
shinhodong_sorted['순번'] = shinhodong_sorted.index + 1
shinhodong_sorted.head()

In [ ]:
# 데이터 정제 파일 저장
shinhodong_sorted.to_csv('../data/shinhodong0.1')

### 5. 송정동 크롤링 및 데이터 정제

In [ ]:
# 라이브러리 로드
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
# 셀레니움 옵션 지정
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

In [ ]:
#송정동
targetUrl = "https://new.land.naver.com/offices?ms=35.0988529,128.8437712,16&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165&articleNo=2409482413"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

In [ ]:
total_items = len(myDriver.find_elements(By.XPATH, '//*[@id="listContents1"]/div/div/div[1]/div'))
total_items

In [ ]:
#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

In [ ]:
number_idx = 0

# Iterate through all items
for i in range(1, total_items + 1):
    # 순번
    number_idx += 1  
    idx_list.append(number_idx)

    # 동적으로 XPath를 생성
    element_xpath = f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]'
    
    # 좌측바 메뉴 클릭
    leftMenu = myDriver.find_element(By.XPATH, value=element_xpath).click()
    time.sleep(4)
    
    # 나머지 코드
    webCurrentStatus = myDriver.page_source
    myParser = BeautifulSoup(webCurrentStatus, "html.parser")
    side_Menu = myParser.select("div.detail_contents.is-article")
    
    for i in side_Menu:
        # ↓ try 블록 내에서 데이터를 추출하는 코드
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle  # 주소 + 매물명 합침
        
        # ↓ 위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

In [ ]:
# 데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

In [ ]:
#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

storagePlace

In [ ]:
for i in range(len(storagePlace)):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
        
    if ":" in storagePlace.loc[i, "면적(㎡)"]:
        storagePlace.loc[i, "면적(㎡)"] = storagePlace.loc[i, "면적(㎡)"].split(":")[2]
    else:
        storagePlace.loc[i, "면적(㎡)"] = ""  # Handle the case where ":" is not present

    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")

    storagePlace.loc[i, "보증금"] = pd.to_numeric(storagePlace.loc[i, "보증금"], errors='coerce')
    storagePlace.loc[i, "월세"] = pd.to_numeric(storagePlace.loc[i, "월세"], errors='coerce')

In [ ]:
# 크롤링 파일 저장
storagePlace.to_csv('../data/songjeongdong0.1')

In [ ]:
# 크롤링 파일 불러오기
songjeongdong = pd.read_csv('../data/songjeongdong0.1')

In [ ]:
# 매매 타입 유형은 가격값을 매매가라는 새로운 칼럼에 기입
sell = songjeongdong[songjeongdong['타입'] == '매매']
songjeongdong.loc[sell.index, '매매가'] = sell['가격']
songjeongdong['매매가'].fillna('-', inplace=True)

In [ ]:
# 구분 칼럼에 송정동 값 기입
songjeongdong['구분']='송정동'
songjeongdong.head(2)

In [ ]:
# 필요한 칼럼만 불러오기
songjeongdong = songjeongdong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
songjeongdong.head(2)

In [ ]:
# 칼럼값 변경
songjeongdong = songjeongdong.rename(columns={'명칭': '기타(세부주소)'})
songjeongdong = songjeongdong.rename(columns={'보증금': '보증금 (단위: 만원)'})
songjeongdong = songjeongdong.rename(columns={'월세': '월세(단위:만원)'})
songjeongdong.head()

In [ ]:
songjeongdong.to_csv('../data/songjeongdong0.2.csv')

### 6. 성북동 크롤링 및 데이터 정제

In [68]:
# 라이브러리 로드
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import pandas as pd
import requests #request 사용을 위해
import re #정규표현을  
import time #timesleep을 위해 사용
import numpy as np
# 셀레니움 옵션 지정
myOption = Options() #옵션객체 변수지정
myOption.add_argument("--start-maximized") #크롬 드라이버 창 최대화
myOption.add_argument("--incognito") #크롬 드라이버 시크릿모드로 진행
myOption.add_experimental_option("excludeSwitches", ["enable-automation"]) #드라이버 시작시 불필요문구 미표시되도록 설정
myOption.add_experimental_option("excludeSwitches", ["enable-logging"]) #터미널상의 불필요문구 미표시되도록 설정
myOption.add_experimental_option("detach", True) #드라이버 자동꺼짐 방지
#드라이버 세팅
myService = Service(ChromeDriverManager().install()) #크롬드라이버 설치
myDriver = webdriver.Chrome(service=myService, options=myOption) #드라이버 서비스 및 옵션 지정

In [69]:
#송정동
targetUrl = "https://new.land.naver.com/offices?ms=35.0849,128.8338,16&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165"
myDriver.get(targetUrl) #타겟링크로부터 정보수신
time.sleep(4)
print(f"{myDriver.current_url} 접속완료")

https://new.land.naver.com/offices?ms=35.096834,128.8435352,16&a=GJCG:TJ:SMS:SG&e=RETAIL&h=165&articleNo=2409482413 접속완료


In [ ]:
total_items = len(myDriver.find_elements(By.XPATH, '//*[@id="listContents1"]/div/div/div[1]/div'))
total_items

In [ ]:
#데이터프레임 필드값으로 활용할 빈리스트 생성
idx_list = list()  #데이터프레임 순번
detailTitle_list = list() #판매제목 
detailPrc_list = list()  #가격 리스트
type_list = list()  #타입 리스트
full_Addr_list = list() #풀 주소 리스트
area_list = list() #면적

In [ ]:
number_idx = 0

# Iterate through all items
for i in range(1, total_items + 1):
    # 순번
    number_idx += 1  
    idx_list.append(number_idx)

    # 동적으로 XPath를 생성
    element_xpath = f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]'
    
    # 좌측바 메뉴 클릭
    leftMenu = myDriver.find_element(By.XPATH, value=element_xpath).click()
    time.sleep(4)
    
    # 나머지 코드
    webCurrentStatus = myDriver.page_source
    myParser = BeautifulSoup(webCurrentStatus, "html.parser")
    side_Menu = myParser.select("div.detail_contents.is-article")
    
    for i in side_Menu:
        # ↓ try 블록 내에서 데이터를 추출하는 코드
        detailTitle = i.select_one("div.detail_fixed > div.main_info_area > div.info_title_wrap").text
        detailType = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.type").text
        detailPrc = i.select_one("div.detail_fixed > div.main_info_area > div.info_article_price > span.price").text
        detailAddr = i.select_one("div.detail_box--summary > table.info_table_wrap > tbody > tr.info_table_item > td.table_td").text
        detailArea = i.select_one("div.main_info_area > div.info_article_feature").text
        full_Addr = detailAddr + " " + detailTitle  # 주소 + 매물명 합침
        
        # ↓ 위에서 추출한 데이터를 빈리스트에 추가
        area_list.append(detailArea)
        detailTitle_list.append(detailTitle)
        type_list.append(detailType)
        detailPrc_list.append(detailPrc)
        full_Addr_list.append(full_Addr)

In [ ]:
# 데이터프레임 생성
storagePlace = pd.DataFrame(
    {
     "순번" : idx_list,
     "명칭" : detailTitle_list,
     "타입" : type_list,
     "가격" : detailPrc_list,
     "주소" : full_Addr_list,
     "면적(㎡)" : area_list
    } 
)

In [ ]:
#전처리를 위해 빈리스트 생성
deposit_list = list()
monthly_fee_list = list()

storagePlace["보증금"] = " " #신규 필드 생성
storagePlace["월세"] = " " #신규 필드 생성
storagePlace["계약면적(㎡)"] = " " #신규 필드 생성
storagePlace["전용면적(㎡)"] = " " #신규 필드 생성
storagePlace["대지면적(㎡)"] = " " #신규 필드 생성
storagePlace = storagePlace[['순번', '명칭', '타입', '면적(㎡)', '계약면적(㎡)', '전용면적(㎡)', '대지면적(㎡)','가격', '보증금', '월세', '주소']]  #필드순번지정

storagePlace

In [ ]:
for i in range(len(storagePlace)):
    if "/" in storagePlace.loc[i,"가격"]:
        storagePlace.loc[i,"보증금"] = storagePlace.loc[i,"가격"].split('/')[0].replace(',','').replace('억','')
        storagePlace.loc[i,"월세"] = storagePlace.loc[i,"가격"].split('/')[1].replace(',','').replace('억','')
    else:
        storagePlace.loc[i,"보증금"] = 0
        storagePlace.loc[i,"월세"] = 0
    
    if storagePlace.loc[i,"주소"][-1] == "층":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
    elif storagePlace.loc[i,"주소"][-1] == "대":
        storagePlace.loc[i,"주소"] = storagePlace.loc[i,"주소"][:-2]
        
    if ":" in storagePlace.loc[i, "면적(㎡)"]:
        storagePlace.loc[i, "면적(㎡)"] = storagePlace.loc[i, "면적(㎡)"].split(":")[2]
    else:
        storagePlace.loc[i, "면적(㎡)"] = ""  # Handle the case where ":" is not present

    if "/" in storagePlace.loc[i,"면적(㎡)"]: 
        storagePlace.loc[i,"계약면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[0].replace("㎡","")
        storagePlace.loc[i,"전용면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].split('/')[1].replace("㎡","")
        storagePlace.loc[i,"대지면적(㎡)"] = "-"
    else:
        storagePlace.loc[i,"계약면적(㎡)"] = "-"
        storagePlace.loc[i,"전용면적(㎡)"] = "-"
        storagePlace.loc[i,"대지면적(㎡)"] = storagePlace.loc[i,"면적(㎡)"].replace("㎡","")

    storagePlace.loc[i, "보증금"] = pd.to_numeric(storagePlace.loc[i, "보증금"], errors='coerce')
    storagePlace.loc[i, "월세"] = pd.to_numeric(storagePlace.loc[i, "월세"], errors='coerce')

In [ ]:
# 크롤링 파일 저장
storagePlace.to_csv('../data/Sungbookdong')

In [ ]:
# 크롤링 저장한 파일 불러오기
sungbookdong = pd.read_csv('../data/Sungbookdong.csv')

In [ ]:
# 컬럼 데이터 오류 값 삭제
for i in range(len(sungbookdong)):
    if sungbookdong.loc[i, '전용면적(㎡)'] == '연 면적':
        sungbookdong = sungbookdong.drop(i)
       
sungbookdong = sungbookdong.reset_index(drop=True)
sungbookdong.head(2)

In [ ]:
# 매매가 칼럼 추가하고 매매 타입 유형에 대해 가격 칼럼 값을 매매가에 기입
sell = sungbookdong[sungbookdong['타입'] == '매매']

sungbookdong.loc[sell.index, '매매가'] = sell['가격']
sungbookdong['매매가'].fillna('-', inplace=True)
sungbookdong.head(2)

In [ ]:
# 구분 컬럼 추가
sungbookdong['구분']='성북동'
sungbookdong.head(2)

In [ ]:
# 필요한 칼럼만 가져오기
sungbookdong = sungbookdong[['순번','구분','타입','계약면적(㎡)','전용면적(㎡)','대지면적(㎡)','매매가','보증금','월세','주소','명칭']]
sungbookdong.head(2)

In [ ]:
sungbookdong = sungbookdong.rename(columns={'명칭': '기타(세부주소)'})
sungbookdong = sungbookdong.rename(columns={'보증금': '보증금 (단위: 만원)'})
sungbookdong = sungbookdong.rename(columns={'월세': '월세(단위:만원)'})
sungbookdong.head()

In [ ]:
# 크롤링한 파일 저장하기
sungbookdong.to_csv('../data/sungbookdong0.1.csv')